In [143]:
from pyscopus import Scopus
import pandas as pd
import requests
import xmltodict, json


from lxml import etree
import requests
import xmltodict, json

In [144]:
print("Ingrese el dato a buscar:")
busqueda = input()

Ingrese el dato a buscar:
medicine


In [145]:
#Consumo API de SCOPUS

In [146]:
key = '0387c7142db3254ed53c09c131ecf8c0'
scopus = Scopus(key)
search_df = scopus.search("ALL("+busqueda+")", count=100)

In [147]:
#Consumo API de DOAJ

In [148]:
url = 'https://www.doaj.org/api/v2/search/articles/all('+busqueda+')?pageSize=100'
headers = {'Content-Type': 'application/json;charset=UTF-8', 'Access-Control-Allow-Origin': '*'}
r = requests.get(url, headers=headers)

In [149]:
#Consumo API de propiedad Inletectual

In [150]:
url_uspto = 'https://developer.uspto.gov/ibd-api/v1/application/publications?searchText='+busqueda+'&start=0&rows=100'
headers_uspto = {'Content-Type': 'application/json;charset=UTF-8', 'Access-Control-Allow-Origin': '*'}
r2_uspto = requests.get(url_uspto, headers=headers_uspto) 

In [151]:
#Consumo API deProQuest

In [152]:
access_token = '8461684a-5a79-407c-9321-93d8d719ddeb'
headers = {
    'Content-type': 'text/xml',
    'Authorization': 'Bearer {}'.format(access_token)
}
data = '<searchRequest><search><query>'+busqueda+'</query><databases><database>medlineprof</database></databases></search><count>100</count></searchRequest>'
resp = requests.post('https://api-dialog.proquest.com/v1/search', headers=headers, data=data)
o = xmltodict.parse(resp.text)
conta = json.dumps(o)
d = json.loads(conta)


In [153]:
#Consumo API de IEEEXPLORE

In [154]:
url_ieeexplore = 'http://ieeexploreapi.ieee.org/api/v1/search/articles?apikey=3dyyvg36jv78dbuu83q3fzzt&format=json&max_records=200&start_record=1&sort_order=asc&sort_field=article_number&querytext='+busqueda
headers_ieeexplore = {'Content-Type': 'application/json;charset=UTF-8', 'Access-Control-Allow-Origin': '*'}
r_ieeexplore = requests.get(url_ieeexplore, headers=headers_ieeexplore)

In [155]:
#Declaracion 
listalinks = list()
listatitulos = list()
listabstract = list()
listautor= list()
listakeywords = list()
listafuente = list()


In [156]:
#Consumo api  de propiedad intelectual sobre titulo,url,abstract,nombre(uspto)
content_ieeexplore = r_ieeexplore.json()
json_ieeexplore = content_ieeexplore['articles']
print(json_ieeexplore[1]['title'])
for i in range(len(json_ieeexplore)):
    print(i)
    try:
        listabstract.append(json_ieeexplore[i]['abstract'])
        #print(json_ieeexplore[i]['abstract'])
        
        listatitulos.append(json_ieeexplore[i]['title'])
        #print(json_ieeexplore[i]['title'])
        
        listalinks.append(json_ieeexplore[i]['pdf_url'])
        #print(json_ieeexplore[i]['pdf_url'])
        
        aux=list()
        for j in range(len(json_ieeexplore[i]['authors']['authors'])):
            aux.append(json_ieeexplore[i]['authors']['authors'][j]['full_name'])
        #print(aux)
        listautor.append(aux)
        try:
            listakeywords.append(json_ieeexplore[i]['index_terms']['ieee_terms']['terms'])
        except:
            listakeywords.append('0')
            
        listafuente.append('ieeexplore')
            
    except:
        print("Error sin abstract")

Reciprocal EMG control of elbow extension by FES
0
1
2
3
4
5
6
7
8
Error sin abstract
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
Error sin abstract
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


In [157]:
print('link',len(listalinks),' title ',len(listatitulos),'abstract ',len(listabstract),' authors ',len(listautor)," keywords ",len(listakeywords),'fuente', len(listafuente))

link 198  title  198 abstract  198  authors  198  keywords  198 fuente 198


In [158]:
#Consumo api  de propiedad intelectual sobre titulo,url,abstract,nombre(uspto)
content_uspto = r2_uspto.json()
json_uspto = content_uspto['results']

for i in range(len(json_uspto)):
    try:

        listakeywords.append(0)
        listafuente.append("uspto")
        
        nivel1=json_uspto[i]['inventionTitle']
        listatitulos.append(nivel1)
        
        nivel2=json_uspto[i]['filelocationURI']
        listalinks.append(nivel2)
        
        nivel3=json_uspto[i]['abstractText'][0]
        listabstract.append(nivel3)
        
        nivel4=json_uspto[i]['assigneeEntityName']
        listautor.append(nivel4)
    except:
        print(" ")


In [159]:
print('link',len(listalinks),' title ',len(listatitulos),'abstract ',len(listabstract),' authors ',len(listautor)," keywords ",len(listakeywords),'fuente', len(listafuente))

link 298  title  298 abstract  298  authors  298  keywords  298 fuente 298


In [160]:
#Consumo api  de doaj sobre titulo,url,abstract,author(uspto),keywords
content = r.json()
#Ingreso al contenido
json = content['results']

for i in range(len(json)):
    try:

        nivel4=json[i]['bibjson']['keywords']
        listakeywords.append(nivel4)
        nivel1=json[i]['bibjson']['link'][0]['url']
        listalinks.append(nivel1)
        nivel2=json[i]['bibjson']['title']
        listatitulos.append(nivel2)
        nivel3=json[i]['bibjson']['abstract']
        listabstract.append(nivel3)
        nivel4=json[i]['bibjson']['author']
        listautor.append(nivel4)
        listafuente.append("doaj")
        
    except:
        print("")

In [161]:
print('link',len(listalinks),' title ',len(listatitulos),'abstract ',len(listabstract),' authors ',len(listautor)," keywords ",len(listakeywords),'fuente', len(listafuente))

link 370  title  370 abstract  370  authors  370  keywords  370 fuente 370


In [162]:
#Consumo api  de SCOPUS sobre titulo,url,abstract,nombre(uspto)
for i in range(len(search_df['scopus_id'])):
    try:
        pub_abs = scopus.retrieve_abstract(search_df['scopus_id'][i])
        listabstract.append(pub_abs['abstract'])
        listalinks.append("https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp="+search_df['scopus_id'][i]+"&origin=inward")
        listatitulos.append(search_df['title'][i])
        listautor.append(search_df['authors'][i])
        listakeywords.append("0")
        listafuente.append("scopus")
    except:
        print(" ")




In [163]:
print('link',len(listalinks),' title ',len(listatitulos),'abstract ',len(listabstract),' authors ',len(listautor)," keywords ",len(listakeywords),'fuente', len(listafuente))

link 445  title  445 abstract  445  authors  445  keywords  445 fuente 445


In [164]:
#Consumo api  de ProQuest sobre titulo,url,abstract,nombre(uspto)
json = d['searchResponse']['result']['doc']

for i in range(len(json)):

        #TITULO
        nivel1=json[i]['field'][1]['value']
        listatitulos.append(nivel1)
        nivel2=json[i]['field'][3]['value']
        response2 = requests.get(nivel2, headers=headers)
        o1 = xmltodict.parse(response2.text)
        
        #LINK
        listalinks.append(o1['Documents']['Literature']['DocInfo']['URL'])
        
        #Fuente
        listafuente.append('ProQuest')
        #Authores
        try:
            
            contributor=o1['Documents']['Literature']['Contributors']['Contributor']
            aux=list()
            for i in range(len(contributor)):
                #print(contributor[i])
                if(contributor[i]['@ContribRole']=='Author'):
                    aux.append(contributor[i]['NormalizedName']+" "+contributor[i]['LastName'])
            listautor.append(aux)
        except: 
             listautor.append("NULL")

        #Keywords
        keywords=o1['Documents']['Literature']['Subjects']['HeadingTerms']['HeadingTerm']
        aux=list()
        for i in range(len(keywords)):
            nivel9=keywords[i]['Heading']
            aux.append(nivel9['#text'])
        listakeywords.append(aux)
        
        #ABSTRACT
        try:
            nivel5= o1['Documents']['Literature']['Abstract']
            listabstract.append(nivel5.get('#text'))
        except: 
            listabstract.append("NULL")

In [165]:
print('link',len(listalinks),' title ',len(listatitulos),'abstract ',len(listabstract),' authors ',len(listautor)," keywords ",len(listakeywords),'fuente', len(listafuente))

link 545  title  545 abstract  545  authors  545  keywords  545 fuente 545


In [166]:
diccionario ={'link':listalinks ,'title':listatitulos,'abstract':listabstract,'authors':listautor,"keywords":listakeywords,'fuente':listafuente}

In [167]:
dataset=pd.DataFrame(diccionario)
print(dataset)

                                                  link  \
0    https://ieeexplore.ieee.org/stamp/stamp.jsp?tp...   
1    https://ieeexplore.ieee.org/stamp/stamp.jsp?tp...   
2    https://ieeexplore.ieee.org/stamp/stamp.jsp?tp...   
3    https://ieeexplore.ieee.org/stamp/stamp.jsp?tp...   
4    https://ieeexplore.ieee.org/stamp/stamp.jsp?tp...   
..                                                 ...   
540  http://dialog.proquest.com/professional/docvie...   
541  http://dialog.proquest.com/professional/docvie...   
542  http://dialog.proquest.com/professional/docvie...   
543  http://dialog.proquest.com/professional/docvie...   
544  http://dialog.proquest.com/professional/docvie...   

                                                 title  \
0    Simulations of foot stability during gait char...   
1     Reciprocal EMG control of elbow extension by FES   
2    Finite element modeling of electromagnetic sig...   
3    Interaction of array of finite electrodes with...   
4    Kinemati

In [168]:
dataset.to_csv('resultado.csv')